In [27]:
from postgres_connect import *

In [28]:
path = 'tunnel-ssh.cer'
# printthis('yes')
conn = get_conn('Yes',path)

Tunnel Started
Connection Made


In [38]:
query_path = 'rider_pip_closure.sql'
with open(query_path, 'r') as file:
    custom_query = file.read()
cursor = conn.cursor()
cursor.execute(custom_query)
# conn.close()

In [39]:
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)

In [33]:
query_path = 'rider_pip_closure_performance.sql'
with open(query_path, 'r') as file:
    custom_query = file.read()
df_pip = pd.read_sql(custom_query,conn)
df_pip.head()

,pip_date,shipping_city,rider_id,rider_name,hub,performance_type,threshold,monday,tuesday,wednesday,thursday,friday,saturday,sunday,avg_fasr,status
0,2024-01-15,Bangalore,karthik_9108468511_02,karthik Gowda,DS BLR BOMM,Morning-Prepaid,0.85,0.84,NaN,0.76,0.86,0.88,NaN,0.66,0.78,fail
1,2024-01-15,Bangalore,karthik_9108468511_02,karthik Gowda,DS BLR BOMM,Morning-COD,0.60,0.40,NaN,1.00,0.75,0.50,NaN,0.00,0.52,fail
2,2024-01-15,Bangalore,karthik_9108468511_02,karthik Gowda,DS BLR BOMM,Evening-Prepaid,0.80,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.00,pass
3,2024-01-15,Bangalore,karthik_9108468511_02,karthik Gowda,DS BLR BOMM,Evening-COD,0.50,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.00,pass
4,2024-01-15,Bangalore,lakshmi_9972726538_02,lakshmi pathi,DS BLR BOMM,Morning-Prepaid,0.85,0.09,NaN,NaN,NaN,NaN,NaN,NaN,0.09,fail


In [40]:
df.head()

,key,pip_date,rider_id,shipping_city,rider_name,hub,vendor_name,rider_age,need_improvement,starting_fasr,closing_fasr,pip_status,pip_result,update_time
0,20240115-Sachain_Anand_Lodhi_7,2024-01-15,Sachain_Anand_Lodhi_7,Bangalore,Sachain Anand Lodhi,DS BLR HBBL,Blitz_Self,88,Morning Prepaid (82%),76.0,83.0,closed,failed,2024-01-22 16:12:32.649883
1,20240115-DS_NCT_DWRK_0313202307_2,2024-01-15,DS_NCT_DWRK_0313202307_2,Delhi,Ashish kumar,DS DEL DWK,PS Manpower,179,Morning Prepaid (69%) Morning COD(52%) Morning...,65.0,81.0,closed,failed,2024-01-22 16:12:32.649883
2,20240115-Atul_Jps_7976431735_023,2024-01-15,Atul_Jps_7976431735_023,Jaipur,Atul_Jps,DS JPR NPRG,JPS Group of Management,18,Morning Prepaid (79%) Morning Prepaid (78%),69.0,68.0,closed,failed,2024-01-22 16:12:32.649883
3,20240115-Swpanil_7039880610_03,2024-01-15,Swpanil_7039880610_03,Mumbai,Swpanil Khandekar,DS BOM KRL,Team HR,14,Morning Prepaid (67%) Morning Prepaid (42%),77.0,82.0,closed,passed,2024-01-22 16:12:32.649883
4,20240115-Pravin_9891013889_03,2024-01-15,Pravin_9891013889_03,Mumbai,Pravin Kumar Paswan,DS BOM KRL,Ambika Logistics,11,Morning Prepaid (82%),78.0,94.0,closed,passed,2024-01-22 16:12:32.649883


In [46]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

# Filter failed riders
failed_riders = df[df['pip_result'] == 'failed']

# Group by city and hub
grouped_data = failed_riders.groupby(['shipping_city', 'hub']).size().reset_index(name='failed_count')

# Group by city to get total rider count
total_riders = df.groupby(['shipping_city', 'hub']).size().reset_index(name='total_count')

# Merge the two dataframes on city
result_df = pd.merge(total_riders, grouped_data, on=['shipping_city','hub'], how='left')

# Calculate the failed rate without decimals
result_df['failed_rate'] = (result_df['failed_count'] / result_df['total_count']) * 100
result_df['failed_rate'] = result_df['failed_rate'].astype(int)

# Email configuration
sender_email = "akshay@blitznow.in"
sender_password = "Outofblitz1!"

# List of recipients
recipients = [
    "akshay@blitznow.in",
]

# Create the message
message = MIMEMultipart()
message['From'] = sender_email
message['To'] = ", ".join(recipients)
message['Subject'] = f"Rider PIP Closure Report Week - {failed_riders['pip_date'].max()}"

# Create an Excel file with two sheets
excel_file_path = 'failed_riders_report.xlsx'

with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    # Write the failed_riders DataFrame to the first sheet
    failed_riders.to_excel(writer, sheet_name='Failed_Riders_Data', index=False)

    # Write the df_pip DataFrame to the second sheet
    df_pip.to_excel(writer, sheet_name='Additional_Sheet', index=False)

# Attach the Excel file
with open(excel_file_path, 'rb') as excel_file:
    attachment = MIMEApplication(excel_file.read(), _subtype='xlsx')
    attachment.add_header('Content-Disposition', 'attachment', filename='pip_closure_report.xlsx')
    message.attach(attachment)

# Create an HTML table for the email body with formatted headers
html_table = result_df.to_html(index=False, border=1, classes='table', escape=False, justify='center')
html_table = html_table.replace('<th>', '<th style="font-weight:bold; background-color:lightblue; text-align:center;">').replace('_', ' ').upper()

# Add the table to the email body
body = f"""\
<html>
  <head></head>
  <body>
    <p>Dear recipients,</p>
    <p>Please find attached the report for RIDER PIP Result for the week - {failed_riders['pip_date'].max()}.</p>
    <p>Summary:</p>
    {html_table}
    <p>Best regards,<br>Your Name</p>
  </body>
</html>
"""

message.attach(MIMEText(body, 'html'))

# Connect to the SMTP server and send the email
with smtplib.SMTP('smtp.gmail.com', 587) as server:
    server.starttls()
    server.login(sender_email, sender_password)
    server.sendmail(sender_email, recipients, message.as_string())

print("Email sent successfully.")

Email sent successfully.


In [42]:
! pip install xlsxwriter


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 3.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
